In [1]:
import numpy as np

from os import listdir
from os.path import isfile, join
import networkx as nx

import dgl
from dgl.nn import RelGraphConv

import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np


from dgl.nn import GraphConv, AvgPooling
from dgl.nn.pytorch import Sequential

from dgl.dataloading import GraphDataLoader
from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset
from sklearn.model_selection import train_test_split

from sklearn.model_selection import KFold, StratifiedKFold

import matplotlib.pyplot as plt

from torch.optim import lr_scheduler

import pickle

from copy import deepcopy

In [2]:
pickle_name = 'nx_url_dataset.pickle'
with open(pickle_name, 'rb') as f:
    nx_train_set, nx_test_set = pickle.load(f)

In [3]:
e_type_to_id = {'title':0, 'link':1}

def add_encoding(dataset):
    for j in range(len(dataset)):
        g = dataset[j][0]
        for e in g.edges:
#             print(g.edges[e[0], e[1]]['link_type'])
            if g.edges[e[0], e[1]]['link_type'] == 'title':
                g.edges[e[0], e[1]]['type'] = 1
                
            else:
                g.edges[e[0], e[1]]['type'] = 0 
                
        node_domains = dict()
        for i in range(len(g.nodes)):
            g.nodes[i]['h'] = torch.tensor([0., 0., 0.])
            g.nodes[i]['h'][g.nodes[i]['level']] = 1.
            if g.nodes[i]['domain'] not in node_domains:
                node_domains[g.nodes[i]['domain']] = [i]
            else:
                node_domains[g.nodes[i]['domain']].append(i)
            
#         print(node_domains)
        for domain in node_domains:
            if len(node_domains[domain]) >1:
                for d1 in node_domains[domain]:
                    for d2 in node_domains[domain]:
                        if d1 == d2:
                            continue
                        g.add_edge(d1, d2)
                        g.edges[d1, d2]['type'] = 2
                        
    return dataset

# add_encoding([train_set[0]])
train_set = add_encoding(nx_train_set)
test_set = add_encoding(nx_test_set)

In [4]:
train_graphs = [(dgl.from_networkx(g, node_attrs=["h"], edge_attrs=['type']), torch.tensor([l]).float()) for g, l in train_set]
train_labels = [l for _, l in train_set]
test_graphs = [(dgl.from_networkx(g, node_attrs=['h'], edge_attrs=['type']), torch.tensor([l]).float()) for g, l in test_set]
test_labels = [l for _, l in test_set]


In [5]:
class RGCN(nn.Module):
    def __init__(self, hidden_features, dropout, num_edge_types):
        super(RGCN, self).__init__()
        self.in_layer = RelGraphConv(3, hidden_features, num_edge_types, activation=nn.ReLU())
        self.dropout1 = nn.Dropout(dropout)
        self.out_layer = RelGraphConv(hidden_features, hidden_features, 3, activation=nn.ReLU())
        self.dropout2 = nn.Dropout(dropout)
        self.global_mean_pool = dgl.nn.pytorch.glob.AvgPooling()
        
        self.fc = nn.Linear(hidden_features, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, g, in_feat, e_types):
        h = self.in_layer(g, in_feat, e_types)
        h = self.dropout1(h)
        h = self.out_layer(g, h, e_types)
        h = self.dropout2(h)
        h = self.global_mean_pool(g, h)
        h = self.fc(h)
        
        return self.sigmoid(h)

In [6]:
def evaluate(model, dataloader):
    model.eval()
    num_correct = 0
    num_tests = 0
    for batched_graph, labels in dataloader:
        pred = torch.round(model(batched_graph, batched_graph.ndata['h'].float(), batched_graph.edata['type']))
        num_correct += (pred==labels).sum().float().item()
        num_tests += len(labels)
    return num_correct / num_tests

def train(model, train_loader, optimizer):
    Loss = nn.BCELoss()
    
    model.train()
    ret_loss = 0
    n_graphs = 0
    for data, target in train_loader:
        output = model(data, data.ndata['h'].float(), data.edata['type'])
        
        loss = Loss(output, target)
        ret_loss += loss.item()/len(target)
        n_graphs += len(target)     
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        return loss.item()
#     return ret_loss/n_graphs

def val_loss(model, loader):
    Loss = nn.BCELoss()
    model.eval()
    ret_loss = 0
    n_graphs = 0
    with torch.no_grad():
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            output = model(data, data.ndata['h'].float(), data.edata['type'])
            loss = Loss(output, target)

            return loss.item()
        


In [7]:
splits=StratifiedKFold(n_splits=5,shuffle=True,random_state=42)
split_idxs = splits.split(np.arange(len(train_graphs)), train_labels)

num_epochs=100
from itertools import product
hidden_size = [4, 8, 16, 32, 64]
dropout = [0., 0.3, 0.5, 0.7]
lr = [0.1, 0.05, 0.01, 0.005]

tg = dgl.batch([g for g,_ in train_graphs])
num_edge_types = tg.edata['type'].max().item() +1
print(num_edge_types)

model_params = list(product(hidden_size, dropout, lr))

# model_params = [[4, 0., 0.1]]
# print(model_params)

3


In [8]:
# for l in splits.split(np.arange(len(train_graphs)), train_labels):
#     print(train_labels[l])

In [ ]:
best_test_acc = 0
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
results = []
for i, (hidden_size, dropout, lr) in enumerate(model_params):
    foldperf = {}
    print(f"model {i}/{len(model_params)}")
    print('Model parameters: hidden_size {}, dropout {}, optimizer lr {}'.format(*model_params[i]))
    model = RGCN(hidden_size, dropout, num_edge_types)
    
    for fold, (train_idx, val_idx) in enumerate(splits.split(np.arange(len(train_graphs)), train_labels)):
        lrs = []


        train_sampler = SubsetRandomSampler(train_idx)
        test_sampler = SubsetRandomSampler(val_idx)
        train_loader = GraphDataLoader(train_graphs, batch_size=1000, sampler=train_sampler)
        test_loader = GraphDataLoader(train_graphs, batch_size=1000, sampler=test_sampler)

        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        history = {'train_loss': [], 'test_loss': [],'train_acc':[],'test_acc':[]}

        for epoch in range(num_epochs):
            train_loss = train(model,train_loader, optimizer)
            train_acc = evaluate(model, train_loader)
            test_loss = val_loss(model,test_loader)
            test_acc = evaluate(model, test_loader)
#             if epoch%10==1:
#                 print("{} Loss train/val:{:.3f} / {:.3f} Acc train/val {:.2f} / {:.2f} %".format(epoch, train_loss,
#                                                                                               test_loss,
#                                                                                               100*train_acc,
#                                                                                               100*test_acc))

            history['train_loss'].append(train_loss)
            history['test_loss'].append(test_loss)
            history['train_acc'].append(train_acc)
            history['test_acc'].append(test_acc)
        foldperf[f'fold{fold+1}'] = history  
    results.append(foldperf)
#     break


model 0/80
Model parameters: hidden_size 4, dropout 0.0, optimizer lr 0.1


C:\Users\ondra\anaconda3\envs\cp\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


1 Loss train/val:30.647 / 30.811 Acc train/val 48.08 / 47.50 %
11 Loss train/val:8.442 / 7.045 Acc train/val 68.59 / 60.00 %
21 Loss train/val:0.570 / 0.591 Acc train/val 65.38 / 60.00 %
31 Loss train/val:0.575 / 0.569 Acc train/val 70.51 / 62.50 %
41 Loss train/val:0.534 / 0.532 Acc train/val 74.36 / 60.00 %
51 Loss train/val:0.509 / 0.506 Acc train/val 75.00 / 60.00 %
61 Loss train/val:0.486 / 0.483 Acc train/val 80.13 / 70.00 %
71 Loss train/val:0.459 / 0.456 Acc train/val 80.77 / 72.50 %
81 Loss train/val:0.443 / 0.441 Acc train/val 79.49 / 70.00 %
91 Loss train/val:0.427 / 0.425 Acc train/val 81.41 / 75.00 %
1 Loss train/val:15.866 / 1.398 Acc train/val 67.52 / 74.36 %
11 Loss train/val:0.614 / 0.561 Acc train/val 77.71 / 69.23 %
21 Loss train/val:0.618 / 0.500 Acc train/val 79.62 / 79.49 %
31 Loss train/val:0.487 / 0.485 Acc train/val 78.98 / 76.92 %
41 Loss train/val:0.472 / 0.470 Acc train/val 80.25 / 82.05 %
51 Loss train/val:0.460 / 0.459 Acc train/val 82.17 / 79.49 %
61 Loss

C:\Users\ondra\anaconda3\envs\cp\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


1 Loss train/val:5.524 / 3.509 Acc train/val 48.08 / 47.50 %
11 Loss train/val:0.664 / 0.660 Acc train/val 51.92 / 52.50 %
21 Loss train/val:0.640 / 0.638 Acc train/val 51.92 / 52.50 %
31 Loss train/val:0.625 / 0.625 Acc train/val 59.62 / 55.00 %
41 Loss train/val:0.616 / 0.614 Acc train/val 59.62 / 50.00 %
51 Loss train/val:0.594 / 0.594 Acc train/val 66.67 / 60.00 %
61 Loss train/val:0.574 / 0.575 Acc train/val 64.74 / 70.00 %
71 Loss train/val:0.561 / 0.554 Acc train/val 65.38 / 62.50 %
81 Loss train/val:0.542 / 0.541 Acc train/val 67.95 / 65.00 %
91 Loss train/val:0.530 / 0.530 Acc train/val 68.59 / 65.00 %
1 Loss train/val:0.577 / 0.579 Acc train/val 64.33 / 71.79 %
11 Loss train/val:0.579 / 0.577 Acc train/val 70.06 / 69.23 %
21 Loss train/val:0.564 / 0.564 Acc train/val 71.34 / 79.49 %
31 Loss train/val:0.558 / 0.557 Acc train/val 72.61 / 79.49 %
41 Loss train/val:0.548 / 0.548 Acc train/val 70.70 / 71.79 %
51 Loss train/val:0.540 / 0.540 Acc train/val 71.34 / 71.79 %
61 Loss tr

...

1 Loss train/val:7.837 / 8.249 Acc train/val 51.92 / 52.50 %
11 Loss train/val:6.891 / 7.689 Acc train/val 64.10 / 70.00 %
21 Loss train/val:7.411 / 6.698 Acc train/val 67.95 / 65.00 %
31 Loss train/val:24.427 / 23.260 Acc train/val 53.21 / 52.50 %
41 Loss train/val:6.158 / 6.772 Acc train/val 68.59 / 65.00 %
51 Loss train/val:7.680 / 7.524 Acc train/val 67.95 / 67.50 %
61 Loss train/val:2.926 / 1.783 Acc train/val 57.69 / 57.50 %
71 Loss train/val:4.427 / 4.357 Acc train/val 71.79 / 67.50 %
81 Loss train/val:4.665 / 4.658 Acc train/val 74.36 / 70.00 %
91 Loss train/val:1.846 / 1.189 Acc train/val 64.74 / 60.00 %
1 Loss train/val:10.824 / 8.989 Acc train/val 56.69 / 53.85 %
11 Loss train/val:1.359 / 0.920 Acc train/val 63.69 / 58.97 %
21 Loss train/val:2.015 / 1.813 Acc train/val 72.61 / 74.36 %
31 Loss train/val:0.593 / 0.809 Acc train/val 72.61 / 76.92 %
41 Loss train/val:0.556 / 0.681 Acc train/val 72.61 / 76.92 %
51 Loss train/val:0.583 / 0.574 Acc train/val 72.61 / 76.92 %
61 Loss

C:\Users\ondra\anaconda3\envs\cp\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


1 Loss train/val:20.421 / 17.646 Acc train/val 51.92 / 52.50 %
11 Loss train/val:0.745 / 0.668 Acc train/val 71.15 / 65.00 %
21 Loss train/val:0.616 / 0.558 Acc train/val 71.15 / 60.00 %
31 Loss train/val:0.608 / 0.571 Acc train/val 67.31 / 60.00 %
41 Loss train/val:0.586 / 0.551 Acc train/val 71.79 / 60.00 %
51 Loss train/val:0.638 / 0.562 Acc train/val 68.59 / 60.00 %
61 Loss train/val:0.600 / 0.559 Acc train/val 68.59 / 62.50 %
71 Loss train/val:0.567 / 0.550 Acc train/val 69.23 / 62.50 %
81 Loss train/val:0.555 / 0.550 Acc train/val 65.38 / 60.00 %
91 Loss train/val:0.536 / 0.540 Acc train/val 65.38 / 60.00 %
1 Loss train/val:0.711 / 0.706 Acc train/val 65.61 / 61.54 %
11 Loss train/val:0.634 / 0.583 Acc train/val 64.97 / 56.41 %
21 Loss train/val:0.581 / 0.555 Acc train/val 69.43 / 64.10 %
31 Loss train/val:1.310 / 0.526 Acc train/val 70.06 / 61.54 %
41 Loss train/val:0.567 / 2.429 Acc train/val 70.70 / 74.36 %
51 Loss train/val:0.550 / 0.531 Acc train/val 70.06 / 64.10 %
61 Loss 

C:\Users\ondra\anaconda3\envs\cp\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


1 Loss train/val:40.537 / 43.359 Acc train/val 48.08 / 47.50 %
11 Loss train/val:43.326 / 43.275 Acc train/val 46.15 / 50.00 %
21 Loss train/val:32.920 / 30.532 Acc train/val 32.69 / 37.50 %
31 Loss train/val:1.647 / 0.745 Acc train/val 51.92 / 55.00 %
41 Loss train/val:0.631 / 0.613 Acc train/val 66.67 / 55.00 %
51 Loss train/val:0.615 / 0.593 Acc train/val 69.87 / 62.50 %
61 Loss train/val:0.601 / 0.567 Acc train/val 71.15 / 62.50 %
71 Loss train/val:0.623 / 0.573 Acc train/val 71.15 / 70.00 %
81 Loss train/val:0.600 / 0.530 Acc train/val 70.51 / 62.50 %
91 Loss train/val:0.549 / 0.551 Acc train/val 73.72 / 70.00 %
1 Loss train/val:0.634 / 0.552 Acc train/val 70.06 / 61.54 %
11 Loss train/val:0.603 / 0.530 Acc train/val 71.34 / 66.67 %
21 Loss train/val:0.606 / 0.501 Acc train/val 75.80 / 69.23 %
31 Loss train/val:0.679 / 0.479 Acc train/val 77.71 / 74.36 %
41 Loss train/val:0.515 / 0.481 Acc train/val 80.89 / 76.92 %
51 Loss train/val:0.539 / 0.478 Acc train/val 75.16 / 74.36 %
61 L

In [ ]:
for i, foldperf in enumerate(results):
    train_loss = np.vstack([foldperf[fold]['train_loss'] for fold in foldperf])
    test_loss = np.vstack([foldperf[fold]['test_loss'] for fold in foldperf])
    train_acc = np.vstack([foldperf[fold]['train_acc'] for fold in foldperf])
    test_acc = np.vstack([foldperf[fold]['test_acc'] for fold in foldperf])
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(9, 9))
    fig.suptitle(f'Model {i}, hidden_size {model_params[i][0]}, dropout {model_params[i][1]}, lr {model_params[i][2]}')
    ax1.set_xlabel("Epoch")
    ax1.set_ylabel("Loss")
    ax1.title.set_text('Training loss')
    ax1.fill_between(range(num_epochs), train_loss.min(0), train_loss.max(0), alpha=0.3, label="train loss <min; max>")
    ax1.fill_between(range(num_epochs), test_loss.min(0), test_loss.max(0), alpha=0.3, label="test loss <min; max>")

    ax1.plot(range(num_epochs), train_loss.mean(0), label='train loss mean')
    ax1.plot(range(num_epochs), test_loss.mean(0), label='test loss mean')
    ax1.legend()
    
    
    
    ax2.title.set_text("Training loss with cross validation")
    ax2.set_xlabel("Epoch")
    ax2.set_ylabel("Accuracy [%]")
    ax2.fill_between(range(num_epochs), train_acc.min(0), train_acc.max(0), alpha=0.2, label="train acc <min; max>")
    ax2.fill_between(range(num_epochs), test_acc.min(0), test_acc.max(0), alpha=0.2, label="test acc <min; max>")

    ax2.plot(range(num_epochs), train_acc.mean(0), label="Mean train acc")

    ax2.plot(range(num_epochs), test_acc.mean(0), label="Mean test acc")
    ax2.legend()

    
    
    ax3.title.set_text('Train loss per fold')
    for l in train_loss:
        ax3.plot(l)
#     for l in train_loss:
#         ax3.plot(l, 'b-', alpha=0.5)
#     for l in test_loss:
#         ax3.plot(l, 'r-', alpha=0.5)
#     ax3.plot(np.convolve(train_loss.mean(0), np.ones(20)/20, mode='valid'), label='train loss')
#     ax3.plot(np.convolve(test_loss.mean(0), np.ones(20)/20, mode='valid'), label='test loss')
    ax3.set_xlabel('epoch')
    ax3.set_ylabel('running average loss')

    ax3.legend()
    
    ax4.title.set_text('Validation loss per fold')
    for l in test_loss:
        ax4.plot(l)
#     ax3.plot(np.convolve(train_loss.mean(0), np.ones(20)/20, mode='valid'), label='train loss')
#     ax3.plot(np.convolve(test_loss.mean(0), np.ones(20)/20, mode='valid'), label='test loss')
    ax4.set_xlabel('epoch')
    ax4.set_ylabel('Accuracy')
    plt.show()


In [ ]:
accs = []
for i, foldperf in enumerate(results):
    accs.append(np.vstack([foldperf[fold]['test_acc'][-1] for fold in foldperf]).mean(0))
best_params = model_params[np.asarray(accs).argmax()]



In [ ]:
hidden_size, dropout, lr = best_params
model = RGCN(hidden_size, dropout, num_edge_types)

train_loader = GraphDataLoader(train_graphs, batch_size=1000)
# test_loader = GraphDataLoader(train_graphs, batch_size=1000)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
history = {'train_loss': [], 'test_loss': [],'train_acc':[],'test_acc':[]}
train_loss = []
train_acc = []

for epoch in range(num_epochs):
    loss = train(model,train_loader,optimizer)
    acc = evaluate(model, train_loader)



    train_loss.append(loss)
    train_acc.append(acc)


In [ ]:
plt.figure()
fig, (ax1, ax2) = plt.subplots(1,2,figsize=(10, 5))
plt.suptitle('Training loss and acc for the best model')
ax1.set_xlabel('eppch')
ax1.set_ylabel('loss')
ax1.plot(train_loss)

ax2.set_xlabel('epoch')
ax2.set_ylabel('Acc [%]')
ax2.plot(train_acc)

plt.show()

In [ ]:
###### test_loader = GraphDataLoader(test_graphs, batch_size=1000)

print('best params: hidden size {}, dropout {}, Adam learning rate{}'.format(*best_params))
acc = evaluate(model, test_loader)
print(f'Model test acc is {acc*100}%')